In [1]:
import pandas as pd
import glob

In [3]:
reviews = pd.read_excel(open('15.06NRautoreduced to 100.xlsx', 'rb'),
              sheet_name='evaluations') 
reviews = reviews[reviews["customer dimension"]!="not classified"]

In [4]:
diccfilepaths = glob.glob("Dictionaries German language/*")


#sentdict = pd.read_csv(diccfilepaths[0])

diccdf1 = pd.read_csv(
diccfilepaths[0], sep="\t",header=None)
diccdf2 = pd.read_csv(
diccfilepaths[1], sep="\t",header=None)


diccdf = pd.concat([diccdf2,diccdf1],ignore_index=True)
    

diccdf[0] = diccdf[0].map(lambda x: x[0:x.find("|")]+",")
diccdf["words"] = (diccdf[0]+diccdf[2]).astype(str)
diccdf.fillna("nan",inplace=True)

IndexError: list index out of range

In [17]:
reviewsfirsttopic = reviews[reviews["customer dimension"]=="Inquery Handling"]
reviews['body'] = reviews["body"].str.lower()
reviewcounts = countsentiments(reviewsfirsttopic,diccdf)


In [25]:
def countsentiments(filteredreviews,listofsentiments):

    wordlist =[]
    countsofwords = []
    iteration = -1
    for row in listofsentiments["words"]:
        iteration = iteration +1
        if row !="nan":
            #print(row)
            wordlist = str(row).split(",")
            shortcount = []
            
            for word in wordlist:
                
                shortcount.append(filteredreviews['body'].str.count(" "+word.lower()+" ").sum())
            countsofwords.append(sum(shortcount))
        if row =="nan":
            singleword = (listofsentiments[0][iteration])[:-1]
            #print("NAN value    " +  singleword)
            countsofwords.append(filteredreviews['body'].str.count(str(" "+singleword.lower()+" ")).sum())

    listofsentiments["counts for topic: "+str(filteredreviews["customer dimension"].iloc[0])] = countsofwords
    return listofsentiments
#for word in diccdf[0]:
    #countsofwords.append(customerevaluations['body'].str.count(word).sum())



In [26]:
dimensions = reviews["customer dimension"].unique()
for dimension in dimensions:
    filteredreviews = reviews[reviews["customer dimension"]==dimension]
    print(dimension)
    reviewcounts = countsentiments(filteredreviews,diccdf)


Transfer
Praise
Telefon
App
Usage
Bank Card
Account Termination
Complain
Fraud
Apple Pay
Inquery
Customer Service
Account Opening
Online Banking
Bank Account
Employee
Costs
Advisery
Fintech Functions
Credit
ATM
Tan


In [27]:
with pd.ExcelWriter('sentimentanalysisfordimensions15.06.xlsx') as writer:  
    reviewcounts.to_excel(writer, sheet_name='Wordlist')